# Intro to NLP

Deriving information from natural language

Could be sequences text or speech

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f8cda249-c4dc-b874-17de-cd43f93dfa4d)


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-07-26 11:43:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-26 11:43:55 (62.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get dataset

Kaggle disaster tweet set



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-07-26 11:43:57--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-26 11:43:57 (150 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize a text dataset

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Shuffle training data

train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index + 5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real)")
  print(f"Text:\n {text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
 @fadelurker @dalinthanelan &lt; right now.

Even after two years there were still refugees camped just south of Redcliffe village and Aidan &gt;

---

Target: 0 (not real)
Text:
 To navigate inevitable upheaval internal audit must lead the way http://t.co/je86VetDxh

---

Target: 1 (real disaster)
Text:
 After the green line derailment my concern for track that looks like this goes up a bit... @cta @CTAFails http://t.co/1uDz0NVOEH

---

Target: 1 (real disaster)
Text:
 RT: 40HourFamine: In #Bangladesh government has not declared floods an emergency. #WorldVision will continue to mÛ_ http://t.co/XqdVghz8G6

---

Target: 0 (not real)
Text:
 @eles_kaylee @Jannellix0 so she puts on a different face for you ?? her heart broke after buying me lava cakes smh

---



### Split training for a validation set as well

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_text, val_text, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                  train_df_shuffled['target'].to_numpy(),
                                                                  test_size=0.1,
                                                                  random_state=42)

In [ ]:
len(train_text), len(train_labels), len(val_text), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_text[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text to numbers

In [ ]:
# Text vectorization(tokenization)


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None)

In [ ]:
x = round(sum([len(i.split()) for i in train_text])/len(train_text))

In [ ]:
x

15

In [ ]:
max_vocab_length = 10000
max_length = x
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(train_text)

In [ ]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_sentence = random.choice(train_text)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:\n{text_vectorizer(random_sentence)}")

Original text:
@CortezEra I dead ass screamed when we signed him I couldn't fucking believe it      

Vectorized version:
[   1    8  115  227  387   45   46 1592  158    8 2149  255  610   15
    0]


In [ ]:
len(text_vectorizer.get_vocabulary())

10000

### Creating an embedding layer

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

In [ ]:
random_sentence = random.choice(train_text)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:\n")

embedding(text_vectorizer([random_sentence]))

Original text:
Tonight is being quite interesting... A few minutes ago the fire system went off and we had to evacuate the building.      

Embedded version:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-1.09613165e-02,  3.22210826e-02,  8.87353346e-03, ...,
          3.10883783e-02, -5.37261367e-05, -4.28328663e-03],
        [-1.19592175e-02,  4.91891764e-02,  3.23994793e-02, ...,
          1.80437006e-02,  3.91310118e-02, -1.99191570e-02],
        [ 3.95756848e-02,  4.39401157e-02, -1.97380781e-03, ...,
         -2.05923915e-02, -1.73143856e-02,  4.02139463e-02],
        ...,
        [-1.54418349e-02,  2.70414017e-02, -1.06879249e-02, ...,
         -2.82532573e-02,  6.74171373e-03, -2.46619582e-02],
        [ 2.22626813e-02,  3.38884704e-02,  2.34860182e-03, ...,
          1.11422539e-02,  3.79298367e-02, -1.39646418e-02],
        [ 1.35909356e-02,  4.97276820e-02,  4.17434089e-02, ...,
         -3.04315574e-02,  2.18077041e-02,  2.70139053e-03]]],
      dtype=float32)>

## Modelling a text dataset

* Model 0 Naive Bayes
* Model 1 Feed-forward neural network
* Model 2 LSTM
* Model 3 GRU
* Model 4 Bidirectional-LSTM
* Model 5 1D Convolutional Neural Network
* Model 6 Tensorflow Hub Pretrained
* Model 7 As above but on 10%

### Model 0: Getting a baseline

Sklearn's Multinomial Naive Bayes TF-IDF formula to convert our words into numbers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

model_0.fit(train_text, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_score = model_0.score(val_text, val_labels)

In [ ]:
print(f"{baseline_score*100:.2f}%")

79.27%


In [ ]:
baseline_preds = model_0.predict(val_text)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def model_scores(y_true, y_pred):
  model_accuracy = accuracy_score(y_true=y_true, y_pred=y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [ ]:
baseline_results = model_scores(val_labels,
                                baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model



In [ ]:
# Create a tensorboard callback

SAVE_DIR = "model_logs"

In [ ]:
#Build model with functional API

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 output (Dense)              (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_1 = model_1.fit(train_text,
                        train_labels,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_1')])

Saving TensorBoard log files to: model_logs/model_1/20220726-114403
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.6126 - accuracy: 0.6929 - val_loss: 0.5365 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4431 - accuracy: 0.8155 - val_loss: 0.4705 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3483 - accuracy: 0.8593 - val_loss: 0.4579 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2852 - accuracy: 0.8911 - val_loss: 0.4677 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2381 - accuracy: 0.9108 - val_loss: 0.4775 - val_accuracy: 0.7835


In [ ]:
model_1_pred_probs = model_1.predict(val_text)

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
model_1_scores = model_scores(val_labels, model_1_preds)

In [ ]:
import numpy as np

In [ ]:
np.array(list(model_1_scores)) > np.array(list(baseline_results))

array([False, False, False, False])

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [ ]:
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [ ]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [ ]:
embed_weights.shape

(10000, 128)

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")

out_v.close()
out_m.close()

### Recurrent Neural Network (RNN)

### Model 2: LST

LSTM = long short term memory

In [ ]:
tf.constant(["Hallo this is a tweet"]).shape

TensorShape([1])

In [ ]:
# Create an LSTM model

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                      

In [ ]:
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_2 = model_2.fit(train_text,
                        train_labels,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_2')])

Saving TensorBoard log files to: model_logs/model_2/20220726-114420
Epoch 1/5
215/215 [==============================] - 12s 11ms/step - loss: 0.2218 - accuracy: 0.9219 - val_loss: 0.6495 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1594 - accuracy: 0.9410 - val_loss: 0.6064 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1348 - accuracy: 0.9515 - val_loss: 0.7413 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1111 - accuracy: 0.9581 - val_loss: 0.7759 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0901 - accuracy: 0.9666 - val_loss: 1.0614 - val_accuracy: 0.7743


In [ ]:
model_2_pred_probs = model_2.predict(val_text)
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_results = model_scores(val_labels, model_2_preds)
model_2_results

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_2_results = model_scores(val_labels, model_2_preds)
model_2_results

{'accuracy': 77.42782152230971,
 'f1': 0.7722311836526509,
 'precision': 0.7759894665484696,
 'recall': 0.7742782152230971}

### Model 3: GRU RNN

In [ ]:

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
# x = layers.LSTM(units=64, return_sequences=True)(x)
# x = layers.GRU(64, dropout=0.2)(x)
# x = layers.Dense(64, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs)

In [ ]:
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_3 = model_3.fit(train_text,
                        train_labels,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_3')])

Saving TensorBoard log files to: model_logs/model_3/20220726-121206
Epoch 1/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0452 - accuracy: 0.9800 - val_loss: 1.4957 - val_accuracy: 0.7467
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0424 - accuracy: 0.9800 - val_loss: 1.3869 - val_accuracy: 0.7507
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0415 - accuracy: 0.9791 - val_loss: 1.6278 - val_accuracy: 0.7520
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0384 - accuracy: 0.9828 - val_loss: 1.6103 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0377 - accuracy: 0.9818 - val_loss: 1.6989 - val_accuracy: 0.7520


### Model 4: Bidirectional LSTM

In [ ]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs)

In [ ]:
model_4.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_20 (Bidirecti  (None, 15, 128)          98816     
 onal)                                                           
                                                                 
 bidirectional_21 (Bidirecti  (None, 128)              74496     
 onal)                                                           
                                                           

In [ ]:
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_4 = model_4.fit(train_text,
                        train_labels,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_4')])

Saving TensorBoard log files to: model_logs/model_4/20220726-123054
Epoch 1/5
215/215 [==============================] - 11s 18ms/step - loss: 0.0913 - accuracy: 0.9667 - val_loss: 1.3455 - val_accuracy: 0.7467
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0444 - accuracy: 0.9794 - val_loss: 1.5125 - val_accuracy: 0.7533
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0399 - accuracy: 0.9822 - val_loss: 1.5693 - val_accuracy: 0.7546
Epoch 4/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0370 - accuracy: 0.9810 - val_loss: 1.5355 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0367 - accuracy: 0.9823 - val_loss: 1.3120 - val_accuracy: 0.7520


### Model 5: Conv1D

In [ ]:
# Test our embedding layer, conv1d layer and max pooling

embedding_test = embedding(text_vectorizer(["this is a flood warning"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)



In [ ]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.10524355, 0.3506496 , 0.31206405, 0.19769254, 0.20759478,
        0.09032463, 0.11349934, 0.32397807, 0.06071264, 0.16530666,
        0.1760705 , 0.16524252, 0.16416475, 0.2535678 , 0.35452026,
        0.17071691, 0.21536839, 0.17813346, 0.2449894 , 0.2659509 ,
        0.2216869 , 0.28165793, 0.15085089, 0.3683449 , 0.19248782,
        0.27810994, 0.34750652, 0.16629592, 0.11591729, 0.03156529,
        0.21412146, 0.20680226]], dtype=float32)>

In [ ]:
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [ ]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  padding='same',
                  activation='relu')(x)
x = layers.Conv1D(filters=64,
                  kernel_size=2,
                  padding='same',
                  activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs, outputs)

In [ ]:
model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_5 = model_5.fit(train_text,
                        train_labels,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_5')])

Saving TensorBoard log files to: model_logs/model_5/20220726-132937
Epoch 1/5
215/215 [==============================] - 3s 9ms/step - loss: 0.0753 - accuracy: 0.9743 - val_loss: 1.1200 - val_accuracy: 0.7402
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0433 - accuracy: 0.9810 - val_loss: 1.2696 - val_accuracy: 0.7480
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0402 - accuracy: 0.9816 - val_loss: 1.2717 - val_accuracy: 0.7454
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0388 - accuracy: 0.9818 - val_loss: 1.2786 - val_accuracy: 0.7454
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0368 - accuracy: 0.9823 - val_loss: 1.4210 - val_accuracy: 0.7454


### Model 6: TensorFlow hub pretrained sentence encoder

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
embed([sample_sentence])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-0.01157024,  0.02485911,  0.02878048, -0.01271502,  0.0397154 ,
         0.08827759,  0.02680984,  0.05589837, -0.01068729, -0.00597293,
         0.00639323, -0.01819519,  0.00030816,  0.09105889,  0.05874645,
        -0.03180628,  0.01512473, -0.05162928,  0.00991364, -0.06865346,
        -0.04209306,  0.02678978,  0.03011007,  0.00321067, -0.00337968,
        -0.04787357,  0.02266722, -0.00985927, -0.04063614, -0.01292094,
        -0.04666385,  0.056303  , -0.03949257,  0.00517688,  0.02495825,
        -0.07014439,  0.02871507,  0.04947682, -0.00633977, -0.08960193,
         0.02807119, -0.00808361, -0.01360602,  0.05998649, -0.10361786,
        -0.05195374,  0.00232955, -0.02332531, -0.03758105,  0.0332773 ,
        -0.00430603, -0.05894247, -0.06101276, -0.02220004, -0.01575761,
        -0.00474413, -0.03515062, -0.04440378, -0.04174353,  0.04943503,
        -0.02740929, -0.04133103,  0.01634344, -0.03713681,  0.07395207,
 

In [ ]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [ ]:
model_6 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(256, activation="relu"),
  layers.Dense(256, activation="relu"),
  layers.Dense(1, activation="sigmoid")                               
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_6.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_27 (Dense)            (None, 256)               131328    
                                                                 
 dense_28 (Dense)            (None, 256)               65792     
                                                                 
 dense_29 (Dense)            (None, 1)                 257       
                                                                 
Total params: 256,995,201
Trainable params: 197,377
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
history_6 = model_6.fit(train_text,
                        train_labels,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_6')])

Saving TensorBoard log files to: model_logs/model_6/20220726-135205
Epoch 1/5
215/215 [==============================] - 5s 14ms/step - loss: 0.4551 - accuracy: 0.7977 - val_loss: 0.4288 - val_accuracy: 0.8123
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3832 - accuracy: 0.8348 - val_loss: 0.4405 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3498 - accuracy: 0.8523 - val_loss: 0.4294 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2951 - accuracy: 0.8784 - val_loss: 0.4398 - val_accuracy: 0.8215
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2311 - accuracy: 0.9104 - val_loss: 0.5067 - val_accuracy: 0.8163


In [ ]:
model_6_pred_probs = model_6.predict(val_text)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_results = model_scores(val_labels, model_6_preds)
model_6_results

{'accuracy': 80.5774278215223,
 'f1': 0.8047076421415382,
 'precision': 0.8065406579811286,
 'recall': 0.8057742782152231}

### Model 7: TF Hub Pretrained USE but with 10% of training data

In [ ]:
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_text_10_percent = tf.constant(train_10_percent['text'])
train_labels_10_percent = tf.constant(train_10_percent['target'])

In [ ]:
len(train_text_10_percent)

761

In [ ]:
model_7 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")                               
])

model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_7 = model_7.fit(train_text_10_percent,
                        train_labels_10_percent,
                        epochs=5,
                        validation_data=(val_text, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_7')])

Saving TensorBoard log files to: model_logs/model_7/20220726-141009
Epoch 1/5
22/22 [==============================] - 2s 44ms/step - loss: 0.6680 - accuracy: 0.6219 - val_loss: 0.6520 - val_accuracy: 0.6640
Epoch 2/5
22/22 [==============================] - 1s 26ms/step - loss: 0.6049 - accuracy: 0.7752 - val_loss: 0.6035 - val_accuracy: 0.7467
Epoch 3/5
22/22 [==============================] - 1s 25ms/step - loss: 0.5370 - accuracy: 0.8073 - val_loss: 0.5499 - val_accuracy: 0.7690
Epoch 4/5
22/22 [==============================] - 1s 25ms/step - loss: 0.4729 - accuracy: 0.8365 - val_loss: 0.5139 - val_accuracy: 0.7782
Epoch 5/5
22/22 [==============================] - 0s 22ms/step - loss: 0.4291 - accuracy: 0.8409 - val_loss: 0.4949 - val_accuracy: 0.7782


In [ ]:
model_7_pred_probs = model_7.predict(val_text)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = model_scores(val_labels, model_7_preds)
model_7_results

{'accuracy': 82.67716535433071,
 'f1': 0.8263150169682,
 'precision': 0.8267925316153663,
 'recall': 0.8267716535433071}

In [ ]:
train_text_10_percent = train_text[:int(0.1*len(train_text))]
train_labels_10_percent = train_labels[:int(0.1*len(train_text))]

In [ ]:
len(train_text_10_percent), len(train_labels_10_percent)

(685, 685)

In [ ]:
pd.Series(train_labels_10_percent).value_counts()

0    406
1    279
dtype: int64